In [73]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import datasets, transforms
# from PIL import Image

In [74]:
BATCH_SIZE=32 #大概需要2G的显存
EPOCHS=5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [75]:
def left_top(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((x, z), axis=1)
    data2 = np.concatenate((z, z), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3
def right_bottom(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((z, z), axis=1)
    data2 = np.concatenate((z, x), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3

In [76]:
def getType(dataset,x):
    index = np.where(dataset[:, 0] == x)
    for i in index:
        Type = dataset[i]
    return Type

In [77]:
def getPixel(x):
    pixel = x[:, 1:]
    return pixel
def getLabel(x):
    label = x[:, 0]
    return label

In [78]:
class train(Dataset):
    def __init__(self):
        mnist_train = pd.read_csv('mnist_train.csv')
        train = mnist_train.values
        
        mnist_train_0=getType(train,0)
        mnist_train_1=getType(train,1)   
        mnist_train_2=getType(train,2)
        mnist_train_3=getType(train,3)   
        
        nrow = 28
        ncol = 28

        # X and label
        x1 = getPixel(mnist_train_1)
        x0 =getPixel(mnist_train_0)
        y1 = getLabel(mnist_train_1)
        y0 = getLabel(mnist_train_0)
        x2 = getPixel(mnist_train_2)
        x3 =getPixel(mnist_train_3)
        y2= getLabel(mnist_train_2)
        y3 = getLabel(mnist_train_3)

        # X reshape to 28 28
               
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        nx2 = x2.shape[0]
        nx3 = x3.shape[0]
       
        x0_reshape = x0.reshape((nx0, nrow, ncol))
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x2_reshape = x2.reshape((nx2, nrow, ncol))
        x3_reshape = x3.reshape((nx3, nrow, ncol))

#------------------------------------------
        left0 = left_top(x0_reshape)
        right1 = right_bottom(x1_reshape) 
        left2=left_top(x2_reshape)
        right3=right_bottom(x3_reshape)
#------------------------------------------
        
        train_data1 = np.concatenate((left0, right1), axis=0)
        train_data2 = np.concatenate((left2, right3), axis=0)
        train_data= np.concatenate((train_data1, train_data2), axis=0)
        
        nn = train_data.shape[0]

        # reshape to 3136 which is 56*56
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        
        yy1 = np.concatenate((y0, y1), axis=0)
        yy2=np.concatenate((y2, y3), axis=0)
        yy=np.concatenate((yy1, yy2), axis=0)
        
        yy=torch.from_numpy(yy)
      
        self.len = yy.shape[0]
        self.x_data = train_data_reshape
        self.y_data = yy


    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
      

    def __len__(self):
        return self.len
dataset1= train()
train_loader = DataLoader(dataset=dataset1,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [79]:
class test(Dataset):
    def __init__(self):
  
        mnist_test = pd.read_csv('mnist_test.csv')
        testtest= mnist_test.values
        
        
        test0=getType(testtest,0)
        test1=getType(testtest,1)
        test2=getType(testtest,2)
        test3=getType(testtest,3)
        nrow = 28
        ncol = 28

        # X and label
        x1 = getPixel(test1)
        x0 =getPixel(test0)
        y1 = getLabel(test1)
        y0 = getLabel(test0)
        x2 = getPixel(test2)
        x3 =getPixel(test3)
        y2 = getLabel(test2)
        y3 = getLabel(test3)



        # X reshape to 28 28
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        nx2 = x2.shape[0]
        nx3 = x3.shape[0]
      
        x0_reshape = x0.reshape((nx0, nrow, ncol))
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x2_reshape = x2.reshape((nx2, nrow, ncol))
        x3_reshape = x3.reshape((nx3, nrow, ncol))

#------------------------------------------
        right0 = right_bottom(x0_reshape)
        left1 = left_top(x1_reshape)
        right2=right_bottom(x2_reshape)
        left3=left_top(x3_reshape)
#------------------------------------------        
        
        
        train_data1 = np.concatenate((right0, left1), axis=0)
        train_data2 = np.concatenate((right2, left3), axis=0)
#         train_data= np.concatenate((train_data1, train_data2), axis=0)
        
        #------------------------------------------
        
        train_data=right0
        

        

        nn = train_data.shape[0]

        
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        
        yy1 = np.concatenate((y0, y1), axis=0)
        yy2=np.concatenate((y2, y3), axis=0)
#         yy=np.concatenate((yy1, yy2), axis=0)
        
         #------------------------------------------
        yy=y0
        yy=torch.from_numpy(yy)
      
        self.len = yy.shape[0]

        self.x_data = train_data_reshape

        self.y_data = yy


    def __getitem__(self, index):
        # img = Image.fromarray(self.x_data[index])
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
dataset2 = test()
test_loader = DataLoader(dataset=dataset2,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [80]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(2880,300)
        self.fc2 = nn.Linear(300,4)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = F.relu(out)   
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # out = F.log_softmax(out,dim=1)
        # out = F.softmax(out,dim=1)
        return out

In [81]:
model = ConvNet()
model = model.to(DEVICE)
model= model.double()
optimizer = optim.Adam(model.parameters())
# criterion = nn.CrossEntropyLoss()

In [82]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # print(data.size(), target.size())
        optimizer.zero_grad()
        output=model(data)
        # print(output)
        #print(target)
        # loss = F.nll_loss(output, target)
        # print(loss)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double())
#             print(target)
#             print(output)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))





In [83]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [3168/24754 (13%)]	Loss: 0.002560
Train Epoch: 1 [6368/24754 (26%)]	Loss: 0.160632
Train Epoch: 1 [9568/24754 (39%)]	Loss: 0.134129
Train Epoch: 1 [12768/24754 (52%)]	Loss: 0.000012
Train Epoch: 1 [15968/24754 (64%)]	Loss: 0.001909
Train Epoch: 1 [19168/24754 (77%)]	Loss: 0.051387
Train Epoch: 1 [22368/24754 (90%)]	Loss: 0.001759

Test set: Average loss: 28.2321, Accuracy: 0/980 (0.0000%)

Train Epoch: 2 [3168/24754 (13%)]	Loss: 0.000002
Train Epoch: 2 [6368/24754 (26%)]	Loss: 0.000001
Train Epoch: 2 [9568/24754 (39%)]	Loss: 0.001865


KeyboardInterrupt: 